## Create Edge List
Creates an edge list for each patch in a provided patch raster and cost surface

In [2]:
#Import libraries
import arcpy
from arcpy.sa import *
import numpy as np
import pandas as pd
from skimage import graph
from matplotlib import pyplot as plt

In [3]:
#Set workspaces
arcpy.env.workspace = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb'
arcpy.env.scratchWorkspace = 'C:/Workspace/PronghornConnectivity/Scratch/Scratch.gdb'
arcpy.env.overwriteOutput = True

In [4]:
def msg(txt):
    print(txt)
    arcpy.AddMessage(txt)
    return

In [8]:
#Get input datasets: Patches and CostSurface
patchRaster = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb/PatchCores'
if not arcpy.Exists(patchRaster):
    print("Cannot locate patch raster")

costRaster = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb/CostSurface'
if not arcpy.Exists(costRaster):
    print("Cannot locate cost surface raster")
    
maxCostDistance = 100000

outRaster = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb/CD{}'
edgeListFN = 'C:/Workspace/PronghornConnectivity/Scratch/EdgeList.csv'

In [9]:
#Get the spatial reference, extent, and lower left coordinates
sr = arcpy.Describe(costRaster).spatialReference
cellSize = arcpy.Describe(costRaster).meanCellWidth
extent = arcpy.Describe(costRaster).extent
llCorner = arcpy.Point(extent.XMin,extent.YMin)

In [10]:
#Create arrays of the patch and core rasters
arrPatch = arcpy.RasterToNumPyArray(patchRaster,
                                    lower_left_corner=llCorner,
                                    nodata_to_value=-9999)

arrCost = arcpy.RasterToNumPyArray(costRaster,
                                   lower_left_corner=llCorner,
                                   nodata_to_value=-9999)

In [11]:
#Create a list of patchIDs
patchIDs = np.unique(arrPatch).tolist()
patchIDs.remove(-9999)

In [13]:
arrList = []
edgeList = []
for patchID in patchIDs[:1]:
    msg("{}...".format(patchID))

    #Reclassify cost in patch cells to zero
    arrCostMod = arrCost.copy()
    arrCostMod[arrPatch == patchID] = 0
    arrCostMod[arrCostMod == -9999] = 100000

    #Create the MCP object (Geometric accounts for diagonals)
    lg = graph.MCP_Geometric(arrCostMod, sampling=(cellSize, cellSize))

    #Get the index of a cell in the current patch ID
    i,j = np.where(arrPatch == patchID)
    startCells = list(zip(i,j))

    #Compute cost distances away from a source
    lcd = lg.find_costs(starts=startCells)[0]
    
    #Write the output to the edgelist
    for toID in patchIDs:
        if toID > patchID:
            edgeList.append((patchID, toID, lcd[arrPatch == toID].min()))

    #Add array to arrList
    arrList.append(lcd)

#Save cost distance raster
#np.save("cd{}".format(patchID),lcd)

#Write the edges to the edgeListFN
np.savetxt(edgeListFN,np.asarray(edgeList),
           delimiter=",", 
           fmt='%d,%d,%2.4f', 
           header=("From,To,Cost"))

1...


In [16]:
#Traceback
tb = lg.traceback([1,1])

In [ ]:
#Merge arrays and save to a file
arrCDs = np.stack(arrList)
np.savetxt('edge.csv',arrCDs)

In [ ]:
#Convert patch ID back to raster
lcd2 = lcd.copy()
lcd2[arrCost == -9999] = np.nan
lcdRaster = arcpy.NumPyArrayToRaster(lcd2,llCorner,cellSize,cellSize)

lcdRaster.save(outRaster.format(patchID))
arcpy.DefineProjection_management(in_dataset=lcdRaster,coor_system=sr);

In [ ]:
arrEdge = np.array()

In [ ]:
#Zonal stats
edges = []
for toID in patchIDs:
    if toID > patchID:
        edges.append((patchID, toID, lcd[arrPatch == toID].min()))

In [ ]:
arrEdge = np.asarray(edges,dtype=np.int)

In [ ]:
arrEdge[3]